# Setting up a simple TESPy model

We want to have a more detailed look at the heat pump. We will take the following steps:

- Set up a thermodynamic model to calculate the COP for oemof-solph
- Run the model at variable ambient temperature levels, calculate the COP
- Export the results to read the into the energy system optimization script

## Flowsheet and important parameters of heat pump

![flowsheet](./workshop/figures/heat_pump.svg)

| parameter description    | model location | model parameter | value | unit |
|:------------------------ |:-------------- |:--------------- | -----:|:---- |
| compressor efficiency    | compressor     | `eta_s`         |    80 | %    |
| heat delivered           | condenser      | `Q`             |   9.1 | kW   |
| working fluid            | 2              | `fluid`         |  R290 |      |
| evaporation temperature  |                | `T`             |     2 | °C   |
| condensation temperatre  | 4              | `T`             |    40 | °C   |

## Python code for the Model

### Preparation

In [ ]:
from tespy.components import Compressor, SimpleHeatExchanger, Valve, CycleCloser
from tespy.networks import Network
from tespy.connections import Connection

In [ ]:
wf = "R290"
nwk = Network(p_unit="bar", T_unit="C", iterinfo=False)

### Setting up the topology

![flowsheet](./workshop/figures/heat_pump.svg)

In [ ]:
cp = Compressor("compressor")
cd = SimpleHeatExchanger("condenser")
va = Valve("valve")
ev = SimpleHeatExchanger("evaporator")

cc = CycleCloser("cycle closer")

c1 = Connection(cc, "out1", ev, "in1")
c2 = Connection(ev, "out1", cp, "in1")
c3 = Connection(cp, "out1", cd, "in1")
c4 = Connection(cd, "out1", va, "in1")
c0 = Connection(va, "out1", cc, "in1")

nwk.add_conns(c1, c2, c3, c4, c0)

### Specification of the parameters

| parameter description    | model location | model parameter | value | unit |
|:------------------------ |:-------------- |:--------------- | -----:|:---- |
| compressor efficiency    | compressor     | `eta_s`         |    80 | %    |
| heat delivered           | condenser      | `Q`             |   9.1 | kW   |
| pressure ratio           |                | `pr`            |   100 | %    |
|                          | evaporator     | `pr`            |   100 | %    |
| working fluid            | 2              | `fluid`         |  R290 |      |
| evaporation temperature  |                | `T`             |     2 | °C   |
| saturated gas stream     |                | `x`             |   100 | %    |
| condensation temperatre  | 4              | `T`             |    40 | °C   |
| saturated liquid stream  |                | `x`             |     0 | %    |

In [ ]:
c2.set_attr(T=2, x=1, fluid={"R290": 1})
c4.set_attr(T=40, x=0)

cp.set_attr(eta_s=0.8)
cd.set_attr(pr=1, Q=-9.1e3)
ev.set_attr(pr=1)

### Run the simulation and calculate COP

$$
COP = \frac{|\dot W_\text{compressor}|}{\dot Q_\text{condenser}}
$$

In [ ]:
nwk.solve("design")
nwk.print_results()

In [ ]:
abs(cd.Q.val) / cp.P.val

### Calibrate to match the datasheet COP

Target COP: 4.9

In [ ]:
cp.set_attr(eta_s=0.75)
nwk.solve("design")
abs(cd.Q.val) / cp.P.val

### Calculate the COP for every ambient temperature value

Variate the evaporation temperature from -15 to 15 (ambient temperature from -10 to 20)

$$
COP = \frac{|\dot W_\text{compressor}|}{\dot Q_\text{condenser}}
$$

$$
COP_\text{Carnot} = \frac{T_4}{T_4 - T_2}
$$

$$
\eta = \frac{COP}{COP_\text{Carnot}}
$$

In [ ]:
import pandas as pd
import numpy as np


temperature_range = np.arange(-10, 21)
results = pd.DataFrame(index=temperature_range, columns=["COP", "COP_carnot"])

cp.eta_s.val

for T in temperature_range:
    c2.set_attr(T=T - 5)
    nwk.solve("design")
    results.loc[T, "COP"] = abs(cd.Q.val) / cp.P.val
    results.loc[T, "COP_carnot"] = c4.T.val_SI / (c4.T.val - c2.T.val)

results["efficiency"] = results["COP"] / results["COP_carnot"]

### Plot COP and efficiency factor

In [ ]:
from matplotlib import pyplot as plt


T_for_eta = 7
eta_const = results.loc[T_for_eta, "efficiency"]

fig, ax = plt.subplots(2, sharex=True)

label = "$\mathrm{COP}_\mathrm{c}$"
ax[0].plot(temperature_range, results["COP_carnot"], label=label)
ax[0].plot(temperature_range, results["COP"], label="$\mathrm{COP}$")
label = "$\mathrm{COP}$: $\eta\left(T=" + str(T_for_eta) + "°C\\right)=" + str(round(eta_const, 3)) + "$"
ax[0].plot(temperature_range, results["COP_carnot"] * eta_const, label=label)
ax[0].set_ylabel("COP")
ax[0].legend()

ax[1].plot(temperature_range, results["efficiency"], color="tab:orange")
ax[1].plot(temperature_range, [eta_const for _ in temperature_range], color="tab:green")
ax[1].set_ylabel("Efficiency factor")

ax[1].set_xlabel("Ambient temperature in °C")

[(a.grid(), a.set_axisbelow(True)) for a in ax];

### Export of COP-temperature lookup

In [ ]:
export = results[["COP"]]
export.index.names = ["temperature"]
export.to_csv("COP-T-tespy.csv")